# L1: Hierarchical Content Generation

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.<br>
<span style="font-size: larger;">To maintain consistency, the notebooks are run with a 'world state' consistent with the video at the start of each notebook.</span></p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

## Creating a World

In [1]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in and understand the world easily.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery and complex.
- You must stay below 3-5 sentences for each description.
"""

In [2]:
world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts, adventurous tales, mysterious people, ancient culture.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


In [3]:
from together import Together
from helper import get_together_api_key,load_env 

client = Together(api_key=get_together_api_key())

output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
)

In [4]:
world_output =output.choices[0].message.content
print(world_output)

World Name: Kyropeia

World Description: Kyropeia is a realm where colossal creatures, known as the "Landstriders", roam the land, their massive bodies serving as the foundation for sprawling cities. These beasts, imbued with ancient magic, have been domesticated by the Kyropeians, who live in symbiosis with them, harnessing their power to sustain their civilization. As the Landstriders migrate, the cities shift, creating an ever-changing landscape of wonder and discovery.


In [5]:
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

## Generating Kingdoms

In [17]:
kingdom_prompt = f"""
Create 4 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, legacy, history of the kingdom and its succesors.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>
Kingdom 4 Name: <KINGDOM NAME>
Kingdom 4 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


In [18]:
output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
)

In [19]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Eldrida" in Kyropeia
Created kingdom "Valtoria" in Kyropeia
Created kingdom "Calonia" in Kyropeia
Created kingdom "Mor'Dhul" in Kyropeia

Kingdom 1 Description: Mor'Dhul is a dark and foreboding kingdom, shrouded in mystery and shadow. Ruled by the reclusive and feared King Xaren, Mor'Dhul is a place of dark magic and ancient secrets, where the people delve deep into the mysteries of the Landstriders and the ancient world. The kingdom's capital, Mor'Dhul City, is a labyrinthine metropolis, its twisting tunnels and dark alleys hiding secrets and dangers at every turn.



Kingdom 2 Description: Mor'Dhul is a dark and foreboding kingdom, built upon the back of a twisted and corrupted Landstrider, the "Shadowstalker". Ruled by the mysterious and feared Queen Xylara, Mor'Dhul is a place of dark magic and shadowy politics, where the inhabitants have mastered the art of manipulation and deception. The kingdom's history is marked by whispers of dark rituals and forbidden knowledge, with its people feared and avoided by the other kingdoms of Kyropeia.


In [10]:
print(f'\nKingdom 3 Description: \
{kingdom["description"]}')


Kingdom 3 Description: Mor'Dhul is a dark and foreboding kingdom, built upon the back of a twisted and corrupted Landstrider, the "Shadowstalker". Ruled by the mysterious and feared Queen Xylara, Mor'Dhul is a place of dark magic and shadowy politics, where the inhabitants have mastered the art of manipulation and deception. The kingdom's history is marked by whispers of dark rituals and forbidden knowledge, with its people feared and avoided by the other kingdoms of Kyropeia.


## Generating Towns

In [24]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom abd world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. About the characteristics and nature of the people living in each town. Make sure that\
    each town name created in each kingdom should differ from other kingdoms \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""


In [25]:
def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.completions.create(
      model="meta-llama/Llama-3-70b-chat-hf",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\n\n'):
        town_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- {town_name} created')
        
        town_description = output.strip().split('\n')[1]\
        .split('Description: ')[1].strip()
        
        town = {
          "name": town_name,
          "description": town_description,
          "world": world['name'],
          "kingdom": kingdom['name']
        }
        towns[town_name] = town
    kingdom["towns"] = towns

In [26]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

town = list(kingdom['towns'].values())[0]
print(f'\nTown 1 Description: \
{town["description"]}')


Creating towns for kingdom: Eldrida...
- Luminaria created
- Aethereia created
- Terraverde created

Creating towns for kingdom: Valtoria...
- Kragnir created
- Tharros created
- Valkraven created

Creating towns for kingdom: Calonia...
- Melodia created
- Chromia created
- Luminaria created

Creating towns for kingdom: Mor'Dhul...
- Shadowhaven created
- Ravenhurst created
- Nightshroud created

Town 1 Description: Located in the shadowy valleys of Mor'Dhul, Shadowhaven is a town of dark alleys and whispering winds. The town is built around the massive, ancient roots of a long-dead Landstrider, its people living in the perpetual twilight beneath the beast's fossilized remains. The town is home to the mysterious Order of the Veiled, a group of shadowy scholars who delve deep into the secrets of dark magic and the ancient world.


## Generating Non-Player Characters (NPC's)

In [27]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different diverse characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, distinct personality, superpowers, weaknesses, background, abilities, and motivation, \
    as well as their deeper pains and desires. \
    Make sure each character feels unique and immersive, with strengths and flaws that shape their journey in the game world.\
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""

In [28]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
        ],
        temperature=0 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for output in npcs_output.split('\n\n'):
        npc_name = output.strip().split('\n')[0]\
        .split('Name: ')[1].strip()
        print(f'- "{npc_name}" created')
        
        npc_description = output.strip().split('\n')[1\
        ].split('Description: ')[1].strip()
        
        npc = {
        "name": npc_name,
        "description": npc_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[npc_name] = npc
    town["npcs"] = npcs

In [29]:
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom
    break


Creating characters for the town of: Luminaria...
- "Kaida Blackwood" created
- "Eirlys Starweaver" created
- "Thrain Stonefist" created

Creating characters for the town of: Aethereia...
- "Kaelin Darkhaven" created
- "Lyrien Starweaver" created
- "Thrain Blackscale" created

Creating characters for the town of: Terraverde...
- "Lyrien Flynn" created
- "Kaida Earthsong" created
- "Arin Vexar" created


In [30]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')


NPC 1 in Terraverde, Eldrida:
Lyrien is a 25-year-old Eldridian mage with short, spiky silver hair and piercing emerald eyes. He wears intricately embroidered robes that reflect his mastery of elemental magic. As a skilled aeromancer, Lyrien can control the winds and skies, summoning storms or gentle breezes with a flick of his wrist. His confidence and charisma make him a natural leader, but his arrogance and tendency to underestimate others can lead to reckless decisions. Lyrien's motivation is to prove himself as the greatest mage in Eldrida, surpassing even Queen Lyra's legendary abilities. His deeper pain is the loss of his family in a tragic accident, which drives his desire for recognition and validation.


## Save the World
>Note: You will save your world state to a file different than the one shown in the video to allow future lessons to be consistent with the video. If later wish to build your own worlds, you will want to load your file rather than the saved file.

In [31]:
import json

def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

#save_world(world, '../shared_data/Kyropeia.json')
save_world(world, 'D:\DLprojects\AIpoweredgame\L1\MyWorld_L1.json') #save to your version